In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [3]:
import malaya_speech
import malaya_speech.config
from malaya_speech.train.model import srgan
from malaya_speech.train.model import enhancement
import tensorflow as tf
import numpy as np






The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.





In [4]:
x = tf.placeholder(tf.float32, (None, 1))
y = tf.placeholder(tf.float32, (None, 1))
partitioned_x = malaya_speech.tf_featurization.pad_and_partition(x, 2048)
partitioned_y = malaya_speech.tf_featurization.pad_and_partition(y, 8192)

with tf.variable_scope('generator') as gen:
    model = srgan.Model(partitioned_x)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [5]:
model.logits, partitioned_y
model.logits.set_shape((None, 8192, 1))

In [6]:
with tf.variable_scope('discriminator') as dis:
    discriminator = srgan.Discriminator(8192)
discriminator

In [7]:
hr_output = discriminator.model(partitioned_y, training = True)
sr_output = discriminator.model(model.logits, training = True)

In [8]:
hr_output, sr_output

(<tf.Tensor 'model/dense_1/Sigmoid:0' shape=(?, 1) dtype=float32>,
 <tf.Tensor 'model_1/dense_1/Sigmoid:0' shape=(?, 1) dtype=float32>)

In [9]:
tf.trainable_variables()

[<tf.Variable 'generator/conv1d/kernel:0' shape=(9, 1, 256) dtype=float32>,
 <tf.Variable 'generator/conv1d/bias:0' shape=(256,) dtype=float32>,
 <tf.Variable 'generator/p_re_lu/alpha:0' shape=(1, 256) dtype=float32>,
 <tf.Variable 'generator/conv1d_1/kernel:0' shape=(3, 256, 256) dtype=float32>,
 <tf.Variable 'generator/conv1d_1/bias:0' shape=(256,) dtype=float32>,
 <tf.Variable 'generator/batch_normalization/gamma:0' shape=(256,) dtype=float32>,
 <tf.Variable 'generator/batch_normalization/beta:0' shape=(256,) dtype=float32>,
 <tf.Variable 'generator/p_re_lu_1/alpha:0' shape=(1, 1) dtype=float32>,
 <tf.Variable 'generator/conv1d_2/kernel:0' shape=(3, 256, 256) dtype=float32>,
 <tf.Variable 'generator/conv1d_2/bias:0' shape=(256,) dtype=float32>,
 <tf.Variable 'generator/batch_normalization_1/gamma:0' shape=(256,) dtype=float32>,
 <tf.Variable 'generator/batch_normalization_1/beta:0' shape=(256,) dtype=float32>,
 <tf.Variable 'generator/conv1d_3/kernel:0' shape=(3, 256, 256) dtype=flo

In [ ]:
hr_output.logits

In [ ]:
logits = tf.reshape(model.logits, (-1, 1))
logits = logits[:tf.shape(y)[0]]
logits

In [ ]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [ ]:
y_lr, sr_lr = malaya_speech.load('../speech/record/savewav_2020-09-06_21-48-43_174842.wav', sr = 11025)
y_hr, sr_hr = malaya_speech.load('../speech/record/savewav_2020-09-06_21-48-43_174842.wav', sr = 44100)

In [ ]:
%%time

y_ = sess.run(logits, feed_dict = {x: np.expand_dims(y_lr, -1), y: np.expand_dims(y_hr, -1)})
y_.shape

In [ ]:
%%time

y_ = sess.run(model.logits, feed_dict = {x: np.expand_dims(y_lr, -1), y: np.expand_dims(y_hr, -1)})
y_.shape

In [ ]:
%%time

y_ = sess.run(partitioned_y, feed_dict = {x: np.expand_dims(y_lr, -1), y: np.expand_dims(y_hr, -1)})
y_.shape

In [ ]:
y_[0].shape

In [ ]:
y_[1].shape

In [ ]:
y_.reshape((-1)).shape

In [ ]:
import IPython.display as ipd

ipd.Audio(y_.reshape((-1)), rate = 44100)

In [ ]:
saver = tf.train.Saver()

In [ ]:
saver.save(sess, 'test/model.ckpt')

In [ ]:
!ls -lh test

In [ ]:
!rm -rf test